In [1]:
import pandas as pd
import sqlite3

In [3]:
#1. Load CSV file
orders=pd.read_csv("orders.csv")
print("Loaded orders",orders.shape)

Loaded orders (10000, 6)


In [5]:
#2. Load JSON file
users=pd.read_json("users.json")
print("Loaded users",users.shape)

Loaded users (3000, 4)


In [35]:
import sqlite3
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    conn.executescript(f.read())
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)

In [37]:
print("restaurant users",restaurants_df.shape)

restaurant users (500, 4)


In [41]:
#Left Join
merged_data = pd.merge(orders, users, on='user_id', how='left')

In [43]:
# Join the current merged_data (left) and restaurants_df (right) on 'restaurant_id'
final_dataset = pd.merge(merged_data, restaurants_df, on='restaurant_id', how='left')

In [45]:
# Save the final DataFrame to a new CSV file
final_dataset.to_csv('final_food_delivery_dataset.csv', index=False)

In [55]:
# Ensure 'order_date' is a datetime object for time series analysis
final_dataset['order_date'] = pd.to_datetime(final_dataset['order_date'])

# Group by date and count orders
daily_orders = final_dataset.groupby(final_dataset['order_date'].dt.date).size()
print("Daily Order Trends:")
print(daily_orders.tail())


Daily Order Trends:
order_date
2023-12-28    31
2023-12-29    37
2023-12-30    26
2023-12-31    29
2024-01-01    27
dtype: int64


C:\Users\vuppa\AppData\Local\Temp\ipykernel_16324\928839241.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_dataset['order_date'] = pd.to_datetime(final_dataset['order_date'])


In [57]:
# Count orders per user and find the top 5
top_users = final_dataset['user_id'].value_counts().head(5)
print("\nTop 5 Users by Order Count:")
print(top_users)


Top 5 Users by Order Count:
user_id
2973    13
1515    12
1337    11
496     11
874     11
Name: count, dtype: int64


In [79]:
# Group by membership type and compare average order value
membership_impact = final_dataset.groupby('membership')['total_amount'].mean()
print("\nAverage Order Value by Membership Type:")
print(membership_impact)


Average Order Value by Membership Type:
membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64


In [83]:
city_revenue = final_dataset.groupby('city')['total_amount'].mean().sort_values(ascending=False)
print("\nAverage Revenue per City:")
print(city_revenue.head())


Average Revenue per City:
city
Chennai      806.202118
Hyderabad    803.985779
Bangalore    802.234308
Pune         792.097914
Name: total_amount, dtype: float64


In [89]:
gold_df = final_dataset[final_dataset['membership'] == 'Gold']


In [92]:
gold_city_revenue = (
    gold_df.groupby('city')['total_amount']
    .sum()
    .reset_index()
    .sort_values('total_amount', ascending=False)
)


In [94]:
print(gold_city_revenue)


        city  total_amount
1    Chennai    1080909.79
3       Pune    1003012.32
0  Bangalore     994702.59
2  Hyderabad     896740.19


In [96]:
cuisine_aov = (
    final_dataset.groupby('cuisine')['total_amount']
      .mean()
      .reset_index(name='avg_order_value')
      .sort_values('avg_order_value', ascending=False)
)

print(cuisine_aov)



   cuisine  avg_order_value
3  Mexican       808.021344
2  Italian       799.448578
1   Indian       798.466011
0  Chinese       798.389020


In [98]:
user_total_spend = (
    final_dataset.groupby('user_id')['total_amount']
      .sum()
      .reset_index(name='total_spent')
)


In [100]:
high_value_users = user_total_spend[user_total_spend['total_spent'] > 1000]


In [102]:
count_users = high_value_users['user_id'].nunique()
print(count_users)


2544


In [106]:
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_dataset['rating_range'] = pd.cut(
    final_dataset['rating'],
    bins=rating_bins,
    labels=rating_labels,
    include_lowest=True
)


In [108]:
rating_revenue = (
    final_dataset.groupby('rating_range')['total_amount']
    .sum()
    .reset_index()
    .sort_values('total_amount', ascending=False)
)

print(rating_revenue)


  rating_range  total_amount
3      4.6–5.0    2197030.75
0      3.0–3.5    2136772.70
2      4.1–4.5    1960326.26
1      3.6–4.0    1717494.41


C:\Users\vuppa\AppData\Local\Temp\ipykernel_16324\1807362404.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_dataset.groupby('rating_range')['total_amount']


In [110]:
gold_city_aov = (
    gold_df.groupby('city')['total_amount']
    .mean()
    .reset_index(name='avg_order_value')
    .sort_values('avg_order_value', ascending=False)
)

print(gold_city_aov)


        city  avg_order_value
1    Chennai       808.459080
2  Hyderabad       806.421034
0  Bangalore       793.223756
3       Pune       781.162243


In [112]:
restaurant_count = (
    final_dataset.groupby('cuisine')['restaurant_id']
    .nunique()
    .reset_index(name='restaurant_count')
)


In [114]:
cuisine_revenue = (
    final_dataset.groupby('cuisine')['total_amount']
    .sum()
    .reset_index(name='total_revenue')
)


In [116]:
cuisine_analysis = restaurant_count.merge(
    cuisine_revenue, on='cuisine'
).sort_values('restaurant_count')

print(cuisine_analysis)


   cuisine  restaurant_count  total_revenue
0  Chinese               120     1930504.65
1   Indian               126     1971412.58
2  Italian               126     2024203.80
3  Mexican               128     2085503.09


In [133]:
total_orders = final_dataset['order_id'].nunique()
gold_orders = final_dataset[final_dataset['membership'] == 'Gold']['order_id'].nunique()


In [135]:
gold_percentage = round((gold_orders / total_orders) * 100)
print(gold_percentage)


50


In [137]:
combo_revenue = (
    final_dataset
    .groupby(['membership', 'cuisine'])['total_amount']
    .sum()
    .reset_index(name='total_revenue')
    .sort_values('total_revenue', ascending=False)
)

print(combo_revenue)


  membership  cuisine  total_revenue
7    Regular  Mexican     1072943.30
6    Regular  Italian     1018424.75
3       Gold  Mexican     1012559.79
2       Gold  Italian     1005779.05
5    Regular   Indian      992100.27
1       Gold   Indian      979312.31
0       Gold  Chinese      977713.74
4    Regular  Chinese      952790.91


In [139]:
final_dataset['order_date'] = pd.to_datetime(final_dataset['order_date'])
final_dataset['quarter'] = final_dataset['order_date'].dt.to_period('Q')


In [141]:
quarterly_revenue = (
    final_dataset.groupby('quarter')['total_amount']
    .sum()
    .reset_index(name='total_revenue')
    .sort_values('total_revenue', ascending=False)
)

print(quarterly_revenue)


  quarter  total_revenue
2  2023Q3     2037385.10
3  2023Q4     2018263.66
0  2023Q1     1993425.14
1  2023Q2     1945348.72
4  2024Q1       17201.50


In [143]:
gold_orders = final_dataset[
    final_dataset['membership'] == 'Gold'
]['order_id'].nunique()

print(gold_orders)


4987


In [145]:
hyderabad_orders = final_dataset[final_dataset['city'] == 'Hyderabad']
hyderabad_revenue = hyderabad_orders['total_amount'].sum()
hyderabad_revenue_rounded = round(hyderabad_revenue)
print(hyderabad_revenue_rounded)


1889367


In [147]:
distinct_users = final_dataset['user_id'].nunique()
print(distinct_users)


2883


In [151]:
high_rated_orders = final_dataset[
    final_dataset['rating'] >= 4.5
]


In [153]:
orders_count = high_rated_orders['order_id'].nunique()
print(orders_count)


3374


In [155]:
gold_city_revenue = (
    gold_df.groupby('city')['total_amount']
    .sum()
    .reset_index(name='total_revenue')
    .sort_values('total_revenue', ascending=False)
)

top_city = gold_city_revenue.iloc[0]['city']
print(top_city)


Chennai


In [157]:
orders_top_city_gold = gold_df[
    gold_df['city'] == top_city
]['order_id'].nunique()

print(orders_top_city_gold)


1337


In [159]:
final_dataset.shape

(10000, 14)